<b> UFC Fight Predictions </b>

This notebook will mostly be a learning exercise for me.  I will be following the tutorial "Train a model to predict a future event" as show in the link https://www.kaggle.com/mdabbert/tutorial-1-train-a-model-to-predict-future-events .  The original author of this work is kaggle user mdabbart".

Once the model has been made, i will plug in data based on the insights i found analysing UFC fight datasets by myself.

In [2]:
### Setting up the environment
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

###The original code imports something called os. 
####I dont know what it is but i will carry ony without it
#### It then prints the files which will be used.

In [3]:
##### This sets the process for dealing with a particular error.
##### The link below explains why it happens
####https://stackoverflow.com/questions/47182183/pandas-chained-assignment-warning-exception-handling

pd.options.mode.chained_assignment = None

In [4]:
###Loading data

df= pd.read_csv("C:/Users/dariu/Desktop/Kaggle/UFC/ufc-master-16-09-20.csv")
df_upcoming=pd.read_csv("C:/Users/dariu/Desktop/Kaggle/UFC/ufc_fn_waterson_hill.csv")

In [5]:
#### cleaning both together

###Creating an empty column
df_combined = df_upcoming.append(df)
df_combined['label'] = ""
df_combined["label"]

###column called mask marks when winner is red, applies 0 if so. and vica versa
mask = df_combined["Winner"]== "Red"
df_combined["label"][mask] = 0
mask = df_combined["Winner"]== "Blue"
df_combined["label"][mask] = 1

###ensures column is numeric
df_combined["label"]= pd.to_numeric(df_combined["label"], errors = "coerce")

###ensures date column is numeric
df_combined["date"]=pd.to_datetime(df["date"])




In [6]:
###Splitting the data frames
label_df = df_combined["label"]

num_upcoming_fights = len(df_upcoming)
df_train = df_combined[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df_combined[:num_upcoming_fights]
label_test = df_combined[:num_upcoming_fights]

### Making sure the sizes are the same
print(len(df_test))
print(len(label_test))

print(len(df_train))
print(len(label_train))

12
12
4355
4355


The above way of doing it is slow and inefficient.  As shown in my previous notebooks, a lambda function works better. E.g df["Winner"].apply(lambda x: 1 if x == "Red" else 0)

In [7]:
###Picking a model and features

my_model = DecisionTreeClassifier(max_depth=5)
my_features = ["win_streak_dif", "height_dif", "reach_dif", "sig_str_dif", "avg_td_dif"]
fighters_test = df_test[["R_fighter","B_fighter"]]
### .copy is used so that the new one tracks any changes
### get dummies is used to edit date into a manageable format

df_train_prepped = df_train[my_features].copy()
df_test_prepped = df_test[my_features].copy()

df_train_prepped = pd.get_dummies(df_train_prepped)
df_test_prepped = pd.get_dummies(df_test_prepped)

###ensuring sets are dummified the same way
df_train_prepped, df_test_prepped = df_train_prepped.align(df_test_prepped, join = "left", axis=1)

####After the join there may be new blank features.
df_test_preppeed = df_test_prepped.fillna(0)

#### making sure the dropped rows match up
label_train_prepped = label_train[label_train.index.isin(df_train_prepped.index)]
label_test_prepped = label_test[label_test.index.isin(df_test_prepped.index)]
fighters_test_prepped = fighters_test[fighters_test.index.isin(df_test_prepped.index)]

####Checking that lengths are the same
print(len(label_train_prepped))
print(len(df_train_prepped))
print(len(df_test_prepped))
print(len(label_test_prepped))
print(len(fighters_test_prepped))

4355
4355
12
12
12


In [8]:
##Training the model
my_model.fit(df_train_prepped, label_train_prepped)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [9]:
###Getting predictions and printing them
predictions = my_model.predict(df_test_prepped)
fighters_array = fighters_test_prepped.to_numpy()
prediction_list = np.array(list(zip(fighters_array, predictions)))
for p in prediction_list:
    if p[1]== 0.0:
        print(f"RED Fighter {p[0][0]} is predicted to win over {p[0][1]}")
    else: 
        print (f'BLUE fighter {p[0][1]} is predicted to win over {p[0][0]}')

RED Fighter Michelle Waterson is predicted to win over Angela Hill
RED Fighter Ottman Azaitar is predicted to win over Khama Worthy
RED Fighter Roxanne Modafferi is predicted to win over Andrea Lee
RED Fighter Ed Herman is predicted to win over Mike Rodriguez
RED Fighter Billy Quarantillo is predicted to win over Kyle Nelson
RED Fighter Julia Avila is predicted to win over Sijara Eubanks
RED Fighter Roosevelt Roberts is predicted to win over Kevin Croom
RED Fighter Alexandr Romanov is predicted to win over Roque Martinez
BLUE fighter Alan Patrick is predicted to win over Bobby Green
RED Fighter Brok Weaver is predicted to win over Jalin Turner
RED Fighter Bryan Barberena is predicted to win over Anthony Ivy
RED Fighter Sabina Mazo is predicted to win over Justine Kish
